In [ ]:
# 首先，导入必要的库
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

df = pd.read_csv('Data/模型历史误差.csv')
plt.rc('font',family='Times New Roman') 
custom_labels = df['Model'].dropna().tolist()
label_idx = 0 

# 为环形柱状图定义多组数据
filtered_df1 = df[df['Variable'] == 'GDP|MER']
group1 = filtered_df1['NORMAL_bias'].tolist()
filtered_df2 = df[df['Variable'] == 'Population']
group2 = filtered_df2['NORMAL_bias'].tolist()
filtered_df3 = df[df['Variable'] == 'Primary Energy']
group3 = filtered_df3['NORMAL_bias'].tolist()
filtered_df4 = df[df['Variable'] == 'Secondary Energy|Electricity|Fossil']
group4 = filtered_df4['NORMAL_bias'].tolist()
filtered_df5 = df[df['Variable'] == 'Secondary Energy|Electricity|Renewables (incl. Biomass)']
group5 = filtered_df5['NORMAL_bias'].tolist()
filtered_df6 = df[df['Variable'] == 'Final Energy|Transportation']
group6 = filtered_df6['NORMAL_bias'].tolist()
filtered_df7 = df[df['Variable'] == 'Final Energy|Industry']
group7 = filtered_df7['NORMAL_bias'].tolist()
filtered_df8 = df[df['Variable'] == 'Final Energy|Residential and Commercial']
group8 = filtered_df8['NORMAL_bias'].tolist()
filtered_df9 = df[df['Variable'] == 'Emissions|CO2']
group9 = filtered_df9['NORMAL_bias'].tolist()
#filtered_df10 = df[df['Variable'] == 'Carbon Sequestration|CCS']
#group10 = filtered_df10['NORMAL_bias'].tolist()

#第二层数据
second_layer_data = df['NORMAL_RMSE'].tolist()

third_layer_data = df['C_NORMAL_RMSE'].tolist()


# 将所有组的数据合并到一个列表中
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9]
#hatches = ['.' if x < 0 else '' for group in groups for x in group]


# 为每组数据定义一个名称，这里使用字母表示
group_names = ["SE1","SE2","ES1","ES2","ES3","ED1","ED2","ED3","CE"]

# 为每组数据定义一个颜色，颜色的数量应该和组的数量一致
group_colors = ['#FFB300', '#FF8F00', '#1CEBC4', '#00C6D0', '#00A1CE', '#cb7bf6', '#7C95FF', '#488CFF', '#EF5350']

# 创建一个画布和子图对象，设置为极坐标系
fig = plt.figure(figsize=(8, 8), dpi=300, facecolor='white')
ax = fig.add_subplot(projection='polar')

# 设置极坐标图的起始点为正北，方向为逆时针
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

radii = []
colors = []

for data, color in zip(groups, group_colors):
# 每组数据后添加分隔符
    radii.append(0)
    colors.append('white')  # 分隔符为白色
    for value in data:
        radii.append(abs(value))  
        if value < 0:
            r, g, b = matplotlib.colors.to_rgb(color)
            colors.append((r, g, b, 0.3))  
        else:
            colors.append(color)  


theta = np.linspace(0.0, 2 * np.pi, len(radii), endpoint=False)  # 计算每个柱子的角度位置
width = 2 * np.pi / len(radii)  # 计算柱子的宽度


# 如果最后一个元素是间隔，则将其移除
if radii[-1] == 0:
    radii.pop()
    colors.pop()


# 确定用于绘图的参数
N = len(radii)  # 柱子的总数
r_lim = 180     # 极坐标的最大半径
scale_lim = 60 # 刻度尺的范围
scale_major = 5 # 主要刻度的步长
bottom = 80     # 柱状图的底部位置
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)  # 计算每个柱子的角度位置
width = 2 * np.pi / (N + 9)  # 计算柱子的宽度

ax.bar(theta, radii, width=width, bottom=bottom, color=colors)


# 画出刻度
def scale(ax, bottom, scale_lim, theta, width):
    t = np.linspace(theta - width / 2, theta + width / 2, 6)
    for i in range(int(bottom), int(bottom + scale_lim + scale_major), scale_major):
        ax.plot(t, [i] * 6, linewidth=0.6, color='gray', alpha=0.6)
# 画出刻度值
def scale_value(ax, bottom, theta, scale_lim):
    for i in range(int(bottom), int(bottom + scale_lim + scale_major), scale_major):
        ax.text(theta, i, f'{i - bottom}', fontsize=6, alpha=1, va='center', ha='center')

s_list = []
g_no = 1
for t, r in zip(theta, radii):
    if r == 0:
        s_list.append(t)
        if t == 0:
            scale_value(ax, bottom, t, scale_lim)
        else:
            scale(ax, bottom, scale_lim, t, width)
    if r != 0:
        t2 = np.rad2deg(t)
        if label_idx < len(custom_labels):  # 检查索引是否在合法范围内
            ax.text(t, r + bottom + scale_major*0.6,
                    custom_labels[label_idx],
                    fontsize=6,
                    rotation=90-t2 if t < np.pi else 270-t2,
                    rotation_mode='anchor',
                    va='center',
                    ha='left' if t < np.pi else 'right',
                    color='black',
                    clip_on=False)
            label_idx += 1  # Increment to the next label
        else:
            print("Label index out of range:", label_idx)

s_list.append( 2*np.pi)

# 绘制第二层数据
def map_values_to_range(values, min_val, max_val):
    min_data, max_data = min(values), max(values)
    # 映射到[min_val, max_val]区间
    return min_val + (max_val - min_val) * (np.array(values) - min_data) / (max_data - min_data)

# 映射second_layer_data到52到74的范围
mapped_radii = map_values_to_range(second_layer_data, 46, 70)

# 绘制散点
for idx, (angle, radius) in enumerate(zip(theta, mapped_radii)):
    if radii[idx] != 0:  # Ensuring there is a corresponding bar
        ax.scatter(angle, radius, color=colors[idx], s=2, zorder=3)

# 绘制第三层数据
mapped_radii2 = map_values_to_range(third_layer_data, 23, 43)
# Create a DataFrame from the mapped data
df = pd.DataFrame({
    'Normalized Second Layer Data': mapped_radii,
    'Normalized Third Layer Data':mapped_radii2
})

# Save the DataFrame to a CSV file
df.to_csv('NormalizedLayersData.csv', index=False)

# 绘制散点
for idx, (angle, radius) in enumerate(zip(theta, mapped_radii2)):
    if radii[idx] != 0:  # 确保有对应的柱子
        ax.scatter(angle, radius, color=colors[idx], s=2, zorder=3)
        
# 绘制环形分界线
theta_sep = np.linspace(0, 2 * np.pi, 100)
# 分界线半径值
r_sep_values = [46, 50, 70, 43, 28.14, 23]
# 分界线样式
line_styles = ['-', '--', '-', '-', '--', '-']
# 分界线颜色
line_colors = ['black', 'black', 'black', 'blue', 'blue', 'blue']

for r_sep, line_style, line_color in zip(r_sep_values, line_styles, line_colors):
    ax.plot(theta_sep, np.ones_like(theta_sep) * r_sep, color=line_color, linewidth=0.5, linestyle=line_style, alpha=0.5)
    #ax.fill_between(theta_sep, 27,np.ones_like(theta_sep) * 47, color='lightblue', alpha=0.2)
    ax.fill_between(theta_sep, 43,np.ones_like(theta_sep) * 46, color='white', alpha=0.8)
    ax.fill_between(theta_sep, 0,np.ones_like(theta_sep) * r_sep, color='lightblue', alpha=0.2)


for i in range(len(s_list)-1):
    t = np.linspace(s_list[i]+width, s_list[i+1]-width, 50)
    ax.plot(t, [bottom-scale_major*0.4]*50, linewidth=1, color='black')
    ax.text(s_list[i]+(s_list[i+1]-s_list[i])/2,
            bottom-scale_major*1.2,
            group_names[i],
            va='center',
            ha='center',
            fontsize=6  )

            
# 设置极坐标图的半径范围，关闭轴线显示
ax.set_rlim(0, bottom+scale_lim+scale_major)
ax.axis('off')

radii_array = np.array(radii)

for idx, angle in enumerate(theta):
    if radii_array[idx] != 0:
        ax.plot([angle, angle], [23, 70], linestyle='--', color='grey', linewidth=0.3, alpha=0.2)


plt.savefig("Figure2.svg",format='svg', dpi=500, bbox_inches='tight')

plt.show()